In [16]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Get embeddings of sentences

Get questions in train.jsonl

In [12]:
from utils import *
import os

dataset_root = '../grade-school-math/grade_school_math/data'

qa_pairs = read_jsonl(os.path.join(dataset_root, 'train.jsonl'))
questions = [e['question'] for e in qa_pairs]
print(f'Number of questions in train dataset: {len(questions)}')

Number of questions in train dataset: 7473


In [13]:
from sentence_transformers import SentenceTransformer
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Device: {device}')

# Load a lightweight transformer model
model = SentenceTransformer('all-distilroberta-v1', device=device)

# Compute sentence embeddings
embeddings = model.encode(questions, convert_to_tensor=True)

# Optional: check tensor shape
print("Embedding shape:", embeddings.shape)

Device: cuda
Embedding shape: torch.Size([7473, 768])


# Group them

In [17]:
group_count = 20
group_numbers = cluster_embeddings(embeddings, num_clusters=group_count)  # Example: 10 clusters

# Print the first 10 group numbers
print(group_numbers[:10])

[9, 0, 4, 5, 5, 19, 14, 14, 10, 0]


In [5]:
add_group_numbers_to_jsonl(original_file_path=os.path.join(dataset_root, 'train.jsonl'),
                           new_file_path=os.path.join(dataset_root, 'train_gr20.jsonl'), group_numbers=group_numbers)

In [15]:
print(count_items_in_groups(group_numbers))

{9: 823, 0: 291, 4: 330, 5: 200, 19: 314, 14: 489, 10: 664, 6: 412, 12: 198, 15: 361, 7: 414, 17: 286, 11: 242, 16: 346, 1: 227, 18: 576, 3: 330, 8: 396, 2: 221, 13: 353}


# Sample from the groups

In [24]:
samples = read_jsonl(os.path.join(dataset_root, 'train_gr20.jsonl'))
sample_per_group = 8

sampled_items = sample_by_group(items=samples,
                                group_numbers=[e['group_number']
                                               for e in samples],
                                samples_per_group=group_count*[sample_per_group])
print(json.dumps(sampled_items, indent=2))
with open(os.path.join(dataset_root, 'train_gr20_samples.json'), 'w') as f:
    # Use json.dumps to serialize the object and write it to the file
    json.dump(sampled_items, f, indent=2)

[
  [
    {
      "question": "For an operations manager job at a company, a person with a degree earns three times the amount paid to a diploma holder for the same position. How much will Jared earn from the company in a year after graduating with a degree if the pay for a person holding a diploma certificate is $4000 per month?",
      "answer": "Since the pay for a person holding a degree is three times the amount paid for a diploma holder, Jared will earn 3*$4000 = $<<4000*3=12000>>12000 per month.\nIn a year with 12 months, Jared will earn a total of 12*12000 = <<12*12000=144000>>144000\n#### 144000",
      "group_number": 0
    },
    {
      "question": "Susan is taking a two week vacation. She works five days a week and has six days of paid vacation. The rest of her workdays will be unpaid vacation time. She gets paid $15 per hour and works 8 hours a day. How much pay will she miss on her vacation?",
      "answer": "Susan works 2 * 5 = <<2*5=10>>10 days in two weeks.\nShe is t